In [4]:
pip install lxml html5lib beautifulsoup4

     |████████████████████████████████| 5.5MB 5.2MB/s eta 0:00:01
     |████████████████████████████████| 122kB 35.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df1 = dfs[0]
df1.head()

# df2 = df1[['Postal Code', 'Borough', 'Neighborhood']]
# df2

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h1>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</h1>

In [4]:
df2 = df1[df1.Borough != 'Not assigned'].reset_index(drop=True)
df2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h1>More than one neighborhood can exist in one postal code area</h1>
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [5]:
df_grouped = df2.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h1>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</h1>

In [6]:
for i, row in df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_grouped.shape

(103, 3)

In [9]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
merged_df = df_grouped.merge(coordinates, on="Postal Code", how="left")
merged_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
CLIENT_ID = '4MGWJGDW3ZLCY13SPCGWHXO0U232XDN34IOONPF4FEA1SAVH'
CLIENT_SECRET = '123HFCKG1Q1VH2JDILYKXOTO1B512T0Y3PSFTN11PXPYZO4K'
VERSION = '20180605'

In [26]:
import requests 
venues_list=[]
radius = 500
LIMIT = 100
for post, lat, lng, borough, neighborhood  in zip(merged_df['Postal Code'], merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        for venue in results:
            venues_list.append((
                post,
                lat, 
                lng,
                borough,
                neighborhood,
                venue['venue']['name'], 
                venue['venue']['location']['lat'], 
                venue['venue']['location']['lng'],  
                venue['venue']['categories'][0]['name']))


In [27]:
venues_df = pd.DataFrame(venues_list)
venues_df.columns = ['Postal Code', 'Latitud', 'Longitude', 'Borough', 'Neighborhood', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
venues_df

,Postal Code,Latitud,Longitude,Borough,Neighborhood,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge",Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek",Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek",Royal Canadian Legion,43.782533,-79.163085,Bar
3,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill",RBC Royal Bank,43.766790,-79.191151,Bank
4,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill",G & G Electronics,43.765309,-79.191537,Electronics Store
...,...,...,...,...,...,...,...,...,...
2127,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",Sheriff's No Frills,43.741696,-79.584379,Grocery Store
2128,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",McDonald's,43.741757,-79.584230,Fast Food Restaurant
2129,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",Dollarama,43.742018,-79.591076,Discount Store
2130,M9W,43.706748,-79.594054,Etobicoke,"Northwest, West Humber - Clairville",Economy Rent A Car,43.708471,-79.589943,Rental Car Location


<h2>Venues per Neighborhood:</h2>

In [35]:
venues_df.rename(columns = {'Postal Code':'PostalCode', 'Venue Category':'VenueCategory'}, inplace = True)
venues_df.groupby(["Neighborhood"]).count()

,PostalCode,Latitud,Longitude,Borough,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,,,
Agincourt,4,4,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23,23,23
...,...,...,...,...,...,...,...,...
"Willowdale, Willowdale East",33,33,33,33,33,33,33,33
"Willowdale, Willowdale West",7,7,7,7,7,7,7,7
Woburn,3,3,3,3,3,3,3,3


<h2>Let's find out how many unique categories can be curated from all the returned venues</h2>

In [37]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 270 uniques categories.


<h1>Analyze Each Neighborhood</h1>

In [43]:
# one hot encoding
venues_df_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_df_onehot['Neighborhood'] = venues_df['Neighborhood']
venues_df_onehot['PostalCode'] = venues_df['PostalCode'] 
venues_df_onehot['Borough'] = venues_df['Borough'] 

# move neighborhood, PostalCode and Borough column to the first column
fixed_columns = list(venues_df_onehot.columns[-3:]) + list(venues_df_onehot.columns[:-3])
venues_df_onehot = venues_df_onehot[fixed_columns]

venues_df_onehot.head()

,Yoga Studio,PostalCode,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,M1B,Scarborough,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,M1C,Scarborough,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,M1C,Scarborough,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,M1E,Scarborough,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,M1E,Scarborough,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h1>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</h1>

In [45]:
toronto_grouped = venues_df_onehot.groupby(["PostalCode", "Borough", "Neighborhood"]).mean().reset_index()
toronto_grouped

,PostalCode,Borough,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,M1B,Scarborough,"Malvern, Rouge",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,Scarborough,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,Scarborough,Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,M9N,York,Weston,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,M9P,Etobicoke,Westmount,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h1>Let's print each neighborhood along with the top 5 most common venues</h1>

In [78]:
sol = []
for index, row in toronto_grouped.iterrows():
    index_aux = 0
    biggest_val = 0
    for row_index in range(3, len(row)):
        if row[row_index] > biggest_val:
            biggest_val = row[row_index]
            index_aux = row_index
    sol.append({
        toronto_grouped.columns[0]: row[0],
        'principalVenue': toronto_grouped.columns[index_aux],
    })
pd.DataFrame(sol)
    

,PostalCode,principalVenue
0,M1B,Fast Food Restaurant
1,M1C,Bar
2,M1E,Bank
3,M1G,Coffee Shop
4,M1H,Athletics & Sports
...,...,...
94,M9N,Park
95,M9P,Pizza Place
96,M9R,Mobile Phone Shop
97,M9V,Grocery Store


<h1>Cluster Neighborhoods</h1>

In [82]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 1, 3, 3, 3, 3], dtype=int32)